In [14]:
import numpy as np
import pandas as pd
import re
import json

In [15]:
df = pd.read_json('../collection/raw_data/apartments.json')
df = df.drop(['recurring_fees', 'one_time_fees'], axis=1)
col = ['address', 'price', 'bed', 'bath', 'area', 'company', 'neighborhood', 'laundry', 'pets', 'parking', 'utilities', 'property_type', 'year_built', 'description', 'images']
df.iloc[1]

address                                         1777 Plymouth Rd
company                                                     None
price                                                   [$1,625]
bed                                                       [1 BR]
bath                                                      [1 BA]
area                                                 [730 Sq Ft]
description    Immediate occupancy! Unique opportunity to liv...
rating                                                       NaN
amenities                                                     []
Name: 1, dtype: object

In [16]:
final_rows = []
for row in df.itertuples():
    address = row[1]
    company = row[2]
    description = row[7]
    rating = row[8]
    amenities = row[9]
    
    for j, s in enumerate(row[3]):
        entry = {'address': address,
                 'company': company, 
                 'price': row[3][j],
                 'bed': row[4][j],
                 'bath': row[5][j],
                 'area': row[6][j],
                 'description': description,
                 'rating': rating,
                 'amenities': amenities
                }
        final_rows.append(entry)

In [17]:
# extract single rows and clean them
final_rows = pd.DataFrame(final_rows)

In [18]:
final_rows.iloc[0:50]

,address,amenities,area,bath,bed,company,description,price,rating
0,2709 N Spurway Dr,[{'Unique Features': ['NewConstruction']}],"2,438 Sq Ft",2½ BAs,3 BRs,None,BRAND NEW! This North Oaks luxury townhome on ...,"$3,300",NaN
1,1777 Plymouth Rd,[],730 Sq Ft,1 BA,1 BR,None,Immediate occupancy! Unique opportunity to liv...,"$1,625",NaN
2,908 Sybil St,"[{'Unique Features': ['No Pets Allowed, Applia...",,1 BA,2 BRs,None,(734) 680-8673 - This is a great building loca...,"$1,625",NaN
3,333 Packard St,[{'Unique Features': ['Cats Allowed - $100 Non...,375 Sq Ft,1 BA,1 BR,None,(734) 996-1991 - Four 1 Bedroom Apartment in b...,"$1,075",NaN
4,513 Krause St,[{'Unique Features': ['Lease Lengths - Contact...,900 Sq Ft,1 BA,1 BR,None,(734) 255-1299 - PRICE LOWERED! Old West Side ...,"$1,900",NaN
5,555 E William St Unit 10E,"[{'Pet Policy': []}, {'Parking': []}, {'Proper...",400 Sq Ft,1 BA,Studio,None,"Renovated 10th floor furnished highrise condo,...","$1,500",NaN
6,2918 Signature Blvd,[],"1,400 Sq Ft",2 BAs,2 BRs,None,"** FOR RENT ** Location, Location, Location! 2...","$1,900",NaN
7,3447 Cummings Dr,"[{'Pet Policy': []}, {'Property Information': ...",,1 BA,1 BR,None,(734) 709-5709 - Furnished bedrooms available ...,$700,NaN
8,City Place Ann Arbor,"[{'Unique Features': ['16' Vaulted Ceiling', '...","2,267 Sq Ft",5 BAs,6 BRs,Campus Village Communities,City Place Ann Arbor offers 6 bedroom townhome...,$920\r\n\t\t\t\t/ Person,5.0
9,City Place Ann Arbor,"[{'Unique Features': ['16' Vaulted Ceiling', '...","2,267 Sq Ft",5 BAs,6 BRs,Campus Village Communities,City Place Ann Arbor offers 6 bedroom townhome...,$970\r\n\t\t\t\t/ Person,5.0


In [19]:
def clean_bed(bed):  
    if 'Studio' not in bed:
        bed = bed[:-3]
    else:
        bed = 0
    return bed

final_rows['bed'] = final_rows['bed'].apply(clean_bed)

In [20]:
def clean_bath(room):
    room = room[:-3]
    if '½' in room:
        room = float(room[0] + '.5')
    else:
        room = float(room)
    return room

final_rows['bath'] = final_rows['bath'].apply(clean_bath)

In [21]:
def clean_area(room):
    room = room[:-5]
    if room != '':
        if len(room) < 7:
            return float(room.replace(',', ''))
        else:
            # mean recorded if price range given
            var = lambda x: (int(x[:x.find('-')].replace(',', '')) + int(x[x.find('-') + 1:].replace(',', ''))) / 2 
            return float(var(room))   
    else:
        return np.nan

final_rows['area'] = final_rows['area'].apply(clean_area)

In [22]:
def clean_price(price):
    price = price.replace(',', '').replace('Person', '').replace('$', '').replace('/', '').strip()
    if len(price) < 7:
        return float(price)
    elif re.search('[Cc]all for [Rr]ent', price) != None:
        return np.nan
    else:
        # mean recorded if price range given
        var = lambda x: (int(x[:x.find('-')].replace(',', '')) + int(x[x.find('-') + 1:].replace(',', ''))) / 2 
        return float(var(price))

final_rows['price'] = final_rows['price'].apply(clean_price)

In [23]:
final_rows

,address,amenities,area,bath,bed,company,description,price,rating
0,2709 N Spurway Dr,[{'Unique Features': ['NewConstruction']}],2438.0,2.5,3,None,BRAND NEW! This North Oaks luxury townhome on ...,3300.0,NaN
1,1777 Plymouth Rd,[],730.0,1.0,1,None,Immediate occupancy! Unique opportunity to liv...,1625.0,NaN
2,908 Sybil St,"[{'Unique Features': ['No Pets Allowed, Applia...",NaN,1.0,2,None,(734) 680-8673 - This is a great building loca...,1625.0,NaN
3,333 Packard St,[{'Unique Features': ['Cats Allowed - $100 Non...,375.0,1.0,1,None,(734) 996-1991 - Four 1 Bedroom Apartment in b...,1075.0,NaN
4,513 Krause St,[{'Unique Features': ['Lease Lengths - Contact...,900.0,1.0,1,None,(734) 255-1299 - PRICE LOWERED! Old West Side ...,1900.0,NaN
5,555 E William St Unit 10E,"[{'Pet Policy': []}, {'Parking': []}, {'Proper...",400.0,1.0,0,None,"Renovated 10th floor furnished highrise condo,...",1500.0,NaN
6,2918 Signature Blvd,[],1400.0,2.0,2,None,"** FOR RENT ** Location, Location, Location! 2...",1900.0,NaN
7,3447 Cummings Dr,"[{'Pet Policy': []}, {'Property Information': ...",NaN,1.0,1,None,(734) 709-5709 - Furnished bedrooms available ...,700.0,NaN
8,City Place Ann Arbor,"[{'Unique Features': ['16' Vaulted Ceiling', '...",2267.0,5.0,6,Campus Village Communities,City Place Ann Arbor offers 6 bedroom townhome...,920.0,5.0
9,City Place Ann Arbor,"[{'Unique Features': ['16' Vaulted Ceiling', '...",2267.0,5.0,6,Campus Village Communities,City Place Ann Arbor offers 6 bedroom townhome...,970.0,5.0


In [24]:
pets, laundry, parking, property_type, year_built, neighborhood, utilities, images = [], [], [], [], [], [], [], []
amenities = final_rows['amenities']
for row in amenities.values:
    row = json.dumps(row)
    
    if re.search('(?<!No )Cats Allowed|(?<!No )Dogs Allowed|considered', row) != None:
        pets.append(1)
    else: 
        pets.append(0)
        
    if re.search('[Ll]aundry|Washer|Dryer', row) != None:
        laundry.append(1)
    else:
        laundry.append(0)
    
    if re.search('parking', row) != None:
        parking.append(1)
    else:
        parking.append(0)
    
    year = re.search('Built in (\d*)', row)
    if year != None:
        year_built.append(year.group(1))
    else:
        year_built.append(np.nan)
        
    property_type.append('apartment')
    neighborhood.append(None)
    utilities.append([])
    images.append([])

In [25]:
final_rows['pets'] = pets
final_rows['laundry'] = laundry
final_rows['parking'] = parking
final_rows['year_built'] = year_built
final_rows['property_type'] = property_type
final_rows['neighborhood'] = neighborhood
final_rows['utilities'] = utilities
final_rows['images'] = images

final_rows = final_rows[col]

In [26]:
final_rows

,address,price,bed,bath,area,company,neighborhood,laundry,pets,parking,utilities,property_type,year_built,description,images
0,2709 N Spurway Dr,3300.0,3,2.5,2438.0,None,None,0,0,0,[],apartment,NaN,BRAND NEW! This North Oaks luxury townhome on ...,[]
1,1777 Plymouth Rd,1625.0,1,1.0,730.0,None,None,0,0,0,[],apartment,NaN,Immediate occupancy! Unique opportunity to liv...,[]
2,908 Sybil St,1625.0,2,1.0,NaN,None,None,1,0,0,[],apartment,NaN,(734) 680-8673 - This is a great building loca...,[]
3,333 Packard St,1075.0,1,1.0,375.0,None,None,1,1,0,[],apartment,NaN,(734) 996-1991 - Four 1 Bedroom Apartment in b...,[]
4,513 Krause St,1900.0,1,1.0,900.0,None,None,1,0,0,[],apartment,NaN,(734) 255-1299 - PRICE LOWERED! Old West Side ...,[]
5,555 E William St Unit 10E,1500.0,0,1.0,400.0,None,None,1,0,0,[],apartment,NaN,"Renovated 10th floor furnished highrise condo,...",[]
6,2918 Signature Blvd,1900.0,2,2.0,1400.0,None,None,0,0,0,[],apartment,NaN,"** FOR RENT ** Location, Location, Location! 2...",[]
7,3447 Cummings Dr,700.0,1,1.0,NaN,None,None,0,0,0,[],apartment,NaN,(734) 709-5709 - Furnished bedrooms available ...,[]
8,City Place Ann Arbor,920.0,6,5.0,2267.0,Campus Village Communities,None,1,0,0,[],apartment,2012,City Place Ann Arbor offers 6 bedroom townhome...,[]
9,City Place Ann Arbor,970.0,6,5.0,2267.0,Campus Village Communities,None,1,0,0,[],apartment,2012,City Place Ann Arbor offers 6 bedroom townhome...,[]


In [27]:
final_rows.to_json(r'../data/apartments.json')